In [18]:
# scipy: para leer archivos .mat
# numpy: para cálculos numéricos
# pandas: para tablas (DataFrame)

import scipy.io
import numpy as np
import pandas as pd


In [19]:
# Cargar el archivo .mat
mat = scipy.io.loadmat('data/B0005.mat')

# Ver qué contiene el archivo
print(mat.keys())


dict_keys(['__header__', '__version__', '__globals__', 'B0005'])


In [20]:
battery = mat['B0005']
print(type(battery))


<class 'numpy.ndarray'>


In [21]:
battery_struct = battery[0, 0]
print(battery_struct.dtype)


[('cycle', 'O')]


In [22]:
cycles = battery_struct['cycle'][0]
print(len(cycles))


616


In [23]:
cycle0 = cycles[0]
print(cycle0.dtype)


[('type', 'O'), ('ambient_temperature', 'O'), ('time', 'O'), ('data', 'O')]


In [24]:
print(cycle0['type'])


['charge']


In [25]:
data = cycle0['data'][0, 0]
print(data.dtype)


[('Voltage_measured', 'O'), ('Current_measured', 'O'), ('Temperature_measured', 'O'), ('Current_charge', 'O'), ('Voltage_charge', 'O'), ('Time', 'O')]


In [26]:
def extraer_datos_bateria(mat, battery_id):
    """
    Convierte un archivo .mat de la NASA en una tabla (DataFrame).
    Solo usa ciclos de descarga (discharge).
    """

    # Accedemos a la estructura principal del archivo
    battery = mat[battery_id][0, 0]

    # Extraemos todos los ciclos
    cycles = battery['cycle'][0]

    filas = []  # aquí guardaremos las filas de la tabla

    # Recorremos cada ciclo
    for num_ciclo, ciclo in enumerate(cycles):

        # Tipo de ciclo: charge / discharge
        tipo = ciclo['type'][0]

        # Solo usamos ciclos de descarga
        if tipo != 'discharge':
            continue

        # Accedemos a los datos del ciclo
        data = ciclo['data'][0, 0]

        # Extraemos los vectores de medición
        tiempo = data['Time'][0]
        voltaje = data['Voltage_measured'][0]
        corriente = data['Current_measured'][0]
        temperatura = data['Temperature_measured'][0]

        # Recorremos cada punto de medición
        for t, v, c, temp in zip(tiempo, voltaje, corriente, temperatura):
            filas.append([
                battery_id,   # ID de la batería
                num_ciclo,    # número de ciclo
                t,            # tiempo
                v,            # voltaje
                c,            # corriente
                temp          # temperatura
            ])

    # Convertimos todo a DataFrame (tabla)
    df = pd.DataFrame(
        filas,
        columns=[
            'bateria',
            'ciclo',
            'tiempo',
            'voltaje',
            'corriente',
            'temperatura'
        ]
    )

    return df


In [27]:
df = extraer_datos_bateria(mat, 'B0005')

df.head()


,bateria,ciclo,tiempo,voltaje,corriente,temperatura
0,B0005,1,0.000,4.191492,-0.004902,24.330034
1,B0005,1,16.781,4.190749,-0.001478,24.325993
2,B0005,1,35.703,3.974871,-2.012528,24.389085
3,B0005,1,53.781,3.951717,-2.013979,24.544752
4,B0005,1,71.922,3.934352,-2.011144,24.731385


In [28]:
df.shape


(50285, 6)

In [31]:
df.to_csv('data/B0005_descarga.csv', index=False)
